In [13]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense 
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers


In [14]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [15]:
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

In [16]:
voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [17]:
glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

In [18]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [19]:
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [20]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         1377900   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         1651

In [21]:
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [27]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [28]:
history = model.fit(x=x_train,y=y_train,batch_size=32,epochs=100)

Epoch 1/100
176/176 [==============================] - 7s 39ms/step - loss: 0.3039 - mean_squared_error: 0.3039 - mean_absolute_error: 0.4336 - mean_absolute_percentage_error: 11.2081
Epoch 2/100
176/176 [==============================] - 8s 44ms/step - loss: 0.2891 - mean_squared_error: 0.2891 - mean_absolute_error: 0.4231 - mean_absolute_percentage_error: 10.9226
Epoch 3/100
176/176 [==============================] - 9s 52ms/step - loss: 0.2897 - mean_squared_error: 0.2897 - mean_absolute_error: 0.4218 - mean_absolute_percentage_error: 10.8976
Epoch 4/100
176/176 [==============================] - 9s 49ms/step - loss: 0.3001 - mean_squared_error: 0.3001 - mean_absolute_error: 0.4249 - mean_absolute_percentage_error: 10.9164
Epoch 5/100
176/176 [==============================] - 8s 43ms/step - loss: 0.2970 - mean_squared_error: 0.2970 - mean_absolute_error: 0.4295 - mean_absolute_percentage_error: 11.0300
Epoch 6/100
176/176 [==============================] - 7s 40ms/step - loss: 0.30

In [32]:
mse = history.history['mean_squared_error']
mae = history.history['mean_absolute_error']
mpe = history.history['mean_absolute_percentage_error']

scores = pd.DataFrame()
scores["MSE"] = mse
scores["MAE"] = mae
scores["% Error"] = mpe

scores

,MSE,MAE,% Error
0,0.303857,0.433553,11.208060
1,0.289113,0.423077,10.922573
2,0.289703,0.421789,10.897609
3,0.300052,0.424854,10.916362
4,0.297047,0.429541,11.029952
...,...,...,...
95,0.185888,0.332464,8.338457
96,0.182806,0.328343,8.145698
97,0.182230,0.329782,8.222215
98,0.181338,0.327712,8.167513
